In [ ]:
import subprocess

def getSampleMatrix():
    script = "./drosChr2L.sh"
    n_samples = 5000
    n_samples_per_run = 100
    is_download = True
    try:
        result = subprocess.run(
            ["bash", script, str(n_samples), str(n_samples_per_run), str(is_download)],
            capture_output=True,
            text=True,
            check=True,
        )

        print("Output:", result.stdout)
        if result.stderr:
            print("Error:", result.stderr)

    except subprocess.CalledProcessError as e:
        print("Script failed with error:", e.stderr)


getSampleMatrix()

Output: Processing file 1 of 1: K.chr1.2480000.2705000.d522c79e1e5749a9abe8f8b61f6784d2dl44ppmy.txt
Running command: ../bin/sBIF -i ../data/folding_input/K.chr1.2480000.2705000.d522c79e1e5749a9abe8f8b61f6784d2dl44ppmy.txt -c chr1 -l ../data/chromosome_sizes.txt -s 2480000 -e 2705000 -ns 5000 -nr 100 -cl K -o ../output -r 5000 -do True -j chr1 -p 50
Parameters: 
Interaction file :../data/folding_input/K.chr1.2480000.2705000.d522c79e1e5749a9abe8f8b61f6784d2dl44ppmy.txt
Chromosome :chr1
Chrom lengths file :../data/chromosome_sizes.txt
Start position:2480000
End position :2705000
Cell line :K
Download :1
Output folder :../output
Resolution :5000
Fiber density :0.2368
Number of samples :5000
Number of samples per run :100
Number of potential sphere points :50
Knock-in distance :80
Maximum trials :100
Number of iteractions :100
Job prefix :chr1
Number of threads :50
Bead diameter: 34.2923
Generating samples ...
Total cost 9.09023 seconds!
Done.



In [13]:
import psycopg2

def query_table():
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        dbname="test",
        user="siyuanzhao",
    )
    cur = conn.cursor()

    query = "SELECT * FROM distance limit 5;"
    cur.execute(query)

    rows = cur.fetchall()
    for row in rows:
        print(row)

    cur.close()
    conn.close()

query_table()


(1, 'K', 'chr1', 900, 2480000, 2705000, 45, [34.2923, 47.7868, 81.0743, 93.7841, 110.611, 123.378, 157.229, 144.587, 174.316, 176.646, 191.878, 214.497, 204.137, 173.419, 149.49, 157.112, 175.932, 195.803, 197.796, 167.718, 138.885, 113.139, 94.4892, 96.8744, 112.141, 141.801, 137.895, 132.019, 149.583, 172.26, 176.438, 163.569, 140.905, 162.068, 173.993, 205.456, 226.917, 230.706, 220.477, 214.44, 226.656, 256.796, 289.273, 299.729, 34.2923, 59.2084, 82.5847, 90.3685, 100.578, 132.871, 115.446, 145.76, 146.318, 164.397, 187.344, 179.575, 149.762, 129.466, 138.444, 160.988, 175.932, 181.186, 149.209, 117.958, 88.9775, 78.4575, 92.8375, 112.739, 136.671, 126.839, 125.035, 149.017, 176.438, 186.921, 174.257, 152.959, 173.993, 191.397, 221.22, 239.987, 240.856, 227.838, 226.656, 243.133, 272.125, 305.221, 314.666, 34.2923, 49.1421, 63.5187, 76.5061, 110.627, 100.702, 130.928, 138.702, 155.773, 182.674, 178.443, 145.948, 129.26, 129.466, 146.353, 162.858, 162.245, 129.99, 102.905, 81.4457,